In [1]:
## !pip install keras

In [2]:
import pandas as pd
import numpy as pd
import pickle
#import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\sbmoi\Anaconda\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\sbmoi\Anaconda\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\sbmoi\Anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\sbmoi\Anaconda\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\sbmoi\Anaconda\lib\site-packages\

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\sbmoi\Anaconda\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\sbmoi\Anaconda\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\sbmoi\Anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\sbmoi\Anaconda\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\sbmoi\Anaconda\lib\site-packages\

AttributeError: _ARRAY_API not found

In [3]:
with open('processed_data_pickle_files_no_smoothing/test_data_batches.pkl', 'rb') as file:
    test = pickle.load(file)
    
with open('processed_data_pickle_files_no_smoothing/train_data_batches.pkl', 'rb') as file:
    train = pickle.load(file)    
    
with open('processed_data_pickle_files_no_smoothing/train_target_values.pkl', 'rb') as file:
    y_train = pickle.load(file)
    
with open('processed_data_pickle_files_no_smoothing/true_rul_values.pkl', 'rb') as file:
    y_test = pickle.load(file)

In [4]:
display(test.shape)
display(train.shape)
display(y_train.shape)
display(y_test.shape)

(100, 30, 14)

(17731, 30, 14)

(17731,)

(100,)

# CNN with Keras by TensorFlow

In [14]:
def make_cnn():
    
    #A sequential model can have different NN layers added. Use a list to define layers, or use add method.
    #Sequential models only work for one input tensor and one output tensor (including for each layer).
    model = Sequential()
    
    #Adds a 1-dimensional convolution layer (only moves in one direction: down each 'sample' of timeseries).
    model.add(Conv1D(filters = 64, #dimension of output space (number of filters)
                     kernel_size = 2, #size of the convolution window
                     activation='relu', #activation function to derive non-linear relationships; default is None.
                     input_shape=(30, 14))) #steps followed by number of features (same order as input)
    
    #Adds a pooling layer to reduce dimensions/computation time - Max or Average is available. 
    #Downsizes the input from the Conv1D layer, so structure is kept but some info is lost.
    model.add(MaxPooling1D(pool_size = 2)) #Number of features considered at once.
    
    #Adds a flattening layer, which makes the 3D data into a 1D array so it's compatible with Dense Layers.
    model.add(Flatten())
    
    #Adds a "dense" layer, which is a regular NN layer. This interprets the output of the previous layers.
    model.add(Dense(50, #First parameter is the output space dimensionality.
                    activation = 'relu')) 
    
    #Adds a second dense layer to reduce the 50 neurons to a single output.
    #This is done seperately for each batch.
    model.add(Dense(1))
    
    #This method configures the model for training. It's where you choose parameters.
    #Other parameter available: loss_weights, metrics, weighted_metrics, etc. 
    model.compile(optimizer = 'adam', loss = 'mse')
    
    return model

In [15]:
display(train.shape)
display(train[0:300].shape)

(17731, 30, 14)

(300, 30, 14)

In [21]:
#Make and train a model using a smaller volume of data (for testing only).
cnn = make_cnn()
cnn.fit(train, y_train, epochs = 100, verbose=0)

In [22]:
y_hat = cnn.predict(test, verbose = 0)

In [23]:
y_hat

array([[ 97.2728   ],
       [113.81402  ],
       [ 19.035753 ],
       [ 75.453766 ],
       [ 73.59957  ],
       [ 97.116486 ],
       [ 88.092186 ],
       [114.96791  ],
       [100.575676 ],
       [109.14251  ],
       [ 68.32842  ],
       [ 88.310646 ],
       [ 73.830055 ],
       [ 84.46639  ],
       [ 82.06845  ],
       [ 84.437485 ],
       [ 27.985167 ],
       [ 24.896643 ],
       [ 63.94509  ],
       [  3.097595 ],
       [ 51.2736   ],
       [127.551186 ],
       [120.32372  ],
       [  9.302646 ],
       [135.80772  ],
       [ 97.2578   ],
       [ 91.80874  ],
       [ 52.08502  ],
       [ 83.412155 ],
       [ 93.14574  ],
       [ -7.97219  ],
       [ 25.518824 ],
       [ 72.028786 ],
       [ -5.2356424],
       [ -6.8644724],
       [  2.3793647],
       [ 26.667389 ],
       [ 32.10423  ],
       [ 99.88973  ],
       [ 19.571905 ],
       [ 20.906416 ],
       [  3.5908973],
       [ 39.519897 ],
       [147.23294  ],
       [ 45.15654  ],
       [ 1

In [28]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(y_test, y_hat)
mse = mean_squared_error(y_test, y_hat)
rmse = np.sqrt(mean_squared_error(y_test, y_hat))
mape = np.mean(np.abs((y_test - y_hat) / y_test)) * 100
r2 = r2_score(y_test, y_hat)

print("MAE: ", mae)
print("MSE: ", mse)
print("RMSE: ", rmse)
print("MAPE: ", mape)
print("R2 Score: ", r2)

MAE:  20.62230623960495
MSE:  687.2638783845198
RMSE:  26.215718155040495
MAPE:  132.1063829045583
R2 Score:  0.6020175218582153


# CNN with PyTorch

In [ ]:
import torch

#This module contains stateful NN layers, are object-oriented and automatically learns manages parameters
import torch.nn as nn 

#This module has stateless layers and adds flexibility over those in torch.nn, requires customization
import torch.nn.functional as F

#Used for images - NOT REQ'D
import torchvision

#Image Transformations - NOT REQ'D
import torchvision.transforms as transforms